# Sasquatch REST Proxy API

This notebook implements the example in the [Sasquatch user guide](https://sasquatch.lsst.io/user-guide/restproxy.html) on using the REST Proxy API to send data to Sasquatch, using the data-int cluster as a sandbox area.


### Get the Kafka cluster ID
First thing you need is to get the cluster ID, this is required to create new topics in Kafka.

In [ ]:
import requests 

sasquatch_rest_proxy_url = "https://data-int.lsst.cloud/sasquatch-rest-proxy"

headers = {"content-type": "application/json"}

r = requests.get(f"{sasquatch_rest_proxy_url}/v3/clusters", headers=headers)

cluster_id = r.json()['data'][0]['cluster_id']

print(cluster_id)

### Create a new Kafka topic
Here we'll use the REST Proxy API on data-int to create a new Kafka topic named `skyFluxMetric` in the `lsst.example` namespace.
Note that if the topic already exists in Kafka the REST Proxy will return an error, but that's ok.

In [ ]:
namespace = "lsst.example"
topic_name = "skyFluxMetric"

# The topic is created with one partition and a replication factor of 3 by default, this configuration is fixed for the Sasquatch Kafka cluster.
topic_config = {  
        "topic_name": f"{namespace}.{topic_name}",
        "partitions_count": 1,
        "replication_factor": 3
    }

headers = {"content-type": "application/json"}

response = requests.post(f"{sasquatch_rest_proxy_url}/v3/clusters/{cluster_id}/topics", json=topic_config, headers=headers)
print(response.text)

### Send data
Now we'll create a payload with the schema and two records based on the example in https://sasquatch.lsst.io/user-guide/restproxy.html#kafka-rest-proxy

In [ ]:
import json

value_schema = json.dumps({"namespace": f"{namespace}", 
                           "type": "record", 
                           "name": f"{topic_name}", 
                           "fields": [{"name": "timestamp", "type": "long"}, 
                                      {"name": "band", "type": "string"}, 
                                      {"name": "instrument", "type": "string", "default": "LSSTCam-imSim"}, 
                                      {"name": "meanSky","type": "float"}, 
                                      {"name": "stdevSky","type": "float"}]
                          })

payload = {
    "value_schema": value_schema,
    "records": [
        {
            "value": {
                "timestamp": 1715967558000000,
                "band": "y",
                "instrument": "LSSTCam-imSim",
                "meanSky": -213.75839364883444,
                "stdevSky": 2328.906118708811,
            }
        },
        {
            "value": {
                "timestamp": 1715967559000000,
                "band": "y",
                "instrument": "LSSTCam-imSim",
                "meanSky": -213.75839364883444,
                "stdevSky": 2328.906118708811,
            }
        }
    ],
}
payload

This is the URL for sending data to our topic


In [ ]:

url = f"{sasquatch_rest_proxy_url}/topics/{namespace}.{topic_name}"
url

Finally, make a POST request to send data:

In [ ]:
headers = {
    "Content-Type": "application/vnd.kafka.avro.v2+json",
    "Accept": "application/vnd.kafka.v2+json",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

You can inspect the data in Kafka using the Kafdrop UI https://data-int.lsst.cloud/kafdrop/topic/lsst.example.skyFluxMetric and also in InlfuxDB using the Chronograf UI https://data-int.lsst.cloud/chronograf